# VGG16 on CIS-PD Spectrograms (w/ Lightning)

In [1]:
%autosave 0

Autosave disabled


In [5]:
# Jupyter Theme switch

#Light theme
#!jt -t grade3

#Dark theme
#!jt -t chesterish


/bin/sh: jt: command not found


In [21]:
# Imports
import os
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import pytorch_lightning as pl
from pytorch_lightning.logging import TensorBoardLogger
from tensorboard import notebook
from argparse import Namespace
import torch.nn as nn
import torchvision as tv
import torchvision.transforms.functional as TF
from torch.nn import functional as F
from torchvision import transforms, models
from torch.utils import data
from torch.utils.data import DataLoader
from PIL import Image
import glob
import random

In [46]:
%run -i Training_Dataset.py

In [13]:
%run -i EvalUtilities.py

ERROR:root:File `'Scoring.py'` not found.


In [51]:
train_ts_dir = 'Data/BEATPDchallenge/BeatPDRealPDupdated/real-pd.training_data_updated/training_data/'
train_label_dir = 'Data/BEATPDchallenge/data_labels-realpd/REAL-PD_Training_Data_IDs_Labels.csv'
ancil_ts_dir = 'Data/BEATPDchallenge/BeatPDRealPDupdated/real-pd.ancillary_data_updated/ancillary_data/'
ancil_label_dir = 'Data/BEATPDchallenge/data_labels-realpd/REAL-PD_Ancillary_Data_IDs_Labels.csv'
sort_by = 'on_off'
dataset_name = 'REAL'

In [52]:
dataset = Training_Dataset(train_ts_dir=train_ts_dir, train_label_dir=train_label_dir,
                 ancil_ts_dir=ancil_ts_dir, ancil_label_dir=ancil_label_dir,
                 sort_by=sort_by, dataset_name=dataset_name, combine_ancil=True)

In [53]:
dataset.create_dictionary()

Creating Dictionary: 100%|██████████| 1081/1081 [02:11<00:00,  8.23it/s]

Issue with 118 measurements enter self.issue_measuements for a list of them.


In [54]:
dataset.run_preprocessing()

Forming Data Lists: 100%|██████████| 3/3 [00:00<00:00, 1813.88it/s]


In [55]:
type(dataset.data_dict[0][0])

tuple

In [3]:
# Directories
root_dir = 'Data/'
spec_dir = os.path.join(root_dir, 'CIS-PD Spectrograms/')

# Training Dataset

In [5]:
### ISSUE: glob not recognizing files #####

# # Making lists with all spectrograms as np arrays
# spec_0 = []
# for f in glob.iglob(spec_dir + '/Medication State 0/*.png'):
#     spec_0.append(np.asarray(Image.open(f)))
    
# spec_1 = []
# for f in glob.iglob(spec_dir + '/Medication State 1/*.png'):
#     spec_1.append(np.asarray(Image.open(f)))
    
# spec_2 = []
# for f in glob.iglob(spec_dir + '/Medication State 2/*.png'):
#     spec_2.append(np.asarray(Image.open(f)))
    
# spec_3 = []
# for f in glob.iglob(spec_dir + '/Medication State 3/*.png'):
#     spec_3.append(np.asarray(Image.open(f)))
    
# spec_4 = []
# for f in glob.iglob(spec_dir + '/Medication State 4/*.png'):
#     spec_4.append(np.asarray(Image.open(f)))

In [4]:
# Works using raw paths
# Making lists with all spectrograms as np arrays
spec_0 = []
for f in glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\zero_centered\Medication State 0\*.png'):
    spec_0.append(np.asarray(Image.open(f)))
    
spec_1 = []
for f in glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\zero_centered\Medication State 1\*.png'):
    spec_1.append(np.asarray(Image.open(f)))
    
spec_2 = []
for f in glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\zero_centered\Medication State 2\*.png'):
    spec_2.append(np.asarray(Image.open(f)))
    
spec_3 = []
for f in glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\zero_centered\Medication State 3\*.png'):
    spec_3.append(np.asarray(Image.open(f)))
    
spec_4 = []
for f in glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\zero_centered\Medication State 4\*.png'):
    spec_4.append(np.asarray(Image.open(f)))

In [5]:
# Functions to prepare spectrograms for model

def crop_center(arr,cropx,cropy):
    y,x,c = arr.shape
    startx = x//2 - cropx//2
    starty = y//2 - cropy//2    
    return arr[starty:starty+cropy, startx:startx+cropx, :]

def drop_4th_channel(arr):
    return arr[:,:,:3]

def SpecPrep(spec):
    spec = crop_center(spec, cropx=670, cropy=272)
    spec = drop_4th_channel(spec)
    return spec

In [6]:
# Preparing spectrograms in each class
for i, spec in enumerate(spec_0):
    spec_0[i] = SpecPrep(spec) 
    
for i, spec in enumerate(spec_1):
    spec_1[i] = SpecPrep(spec) 
    
for i, spec in enumerate(spec_2):
    spec_2[i] = SpecPrep(spec)
    
for i, spec in enumerate(spec_3):
    spec_3[i] = SpecPrep(spec)

for i, spec in enumerate(spec_4):
    spec_4[i] = SpecPrep(spec) 

In [7]:
###### ISSUE: same as above #######


# # Making lists with all of the measurement IDs
# id_list_0 = list(map(os.path.basename,glob.iglob(spec_dir + '/Medication State 0/*.png')))
# for i, ID in enumerate(id_list_0):
#     id_list_0[i] = os.path.splitext(ID)[0]
    
# id_list_1 = list(map(os.path.basename,glob.iglob(spec_dir + '/Medication State 1/*.png')))
# for i, ID in enumerate(id_list_1):
#     id_list_0[i] = os.path.splitext(ID)[0]
    
# id_list_2 = list(map(os.path.basename,glob.iglob(spec_dir + '/Medication State 2/*.png')))
# for i, ID in enumerate(id_list_2):
#     id_list_0[i] = os.path.splitext(ID)[0]
    
# id_list_3 = list(map(os.path.basename,glob.iglob(spec_dir + '/Medication State 3/*.png')))
# for i, ID in enumerate(id_list_3):
#     id_list_0[i] = os.path.splitext(ID)[0]
    
# id_list_4 = list(map(os.path.basename,glob.iglob(spec_dir + '/Medication State 4/*.png')))
# for i, ID in enumerate(id_list_4):
#     id_list_0[i] = os.path.splitext(ID)[0]
    

In [10]:
# Works using raw paths
# Making lists with all of the measurement IDs
id_list_0 = list(map(os.path.basename,glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\zero_centered\Medication State 0\*.png')))
for i, ID in enumerate(id_list_0):
    id_list_0[i] = os.path.splitext(ID)[0]
    
id_list_1 = list(map(os.path.basename,glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\zero_centered\Medication State 1\*.png')))
for i, ID in enumerate(id_list_1):
    id_list_0[i] = os.path.splitext(ID)[0]
    
id_list_2 = list(map(os.path.basename,glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\zero_centered\Medication State 2\*.png')))
for i, ID in enumerate(id_list_2):
    id_list_0[i] = os.path.splitext(ID)[0]
    
id_list_3 = list(map(os.path.basename,glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\zero_centered\Medication State 3\*.png')))
for i, ID in enumerate(id_list_3):
    id_list_0[i] = os.path.splitext(ID)[0]
    
id_list_4 = list(map(os.path.basename,glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\zero_centered\Medication State 4\*.png')))
for i, ID in enumerate(id_list_4):
    id_list_0[i] = os.path.splitext(ID)[0]
    

In [11]:
# Storing data as tuple with their respective measurement_id and label
def AttachMetaData(spec_list, id_list, medstate):
    for i, (spec, ID) in enumerate(zip(spec_list, id_list)):
        spec_list[i] = (spec, ID, medstate)

In [12]:
AttachMetaData(spec_list=spec_0, id_list=id_list_0, medstate=0)
AttachMetaData(spec_list=spec_1, id_list=id_list_1, medstate=1)
AttachMetaData(spec_list=spec_2, id_list=id_list_2, medstate=2)
AttachMetaData(spec_list=spec_3, id_list=id_list_3, medstate=3)
AttachMetaData(spec_list=spec_4, id_list=id_list_4, medstate=4)

In [13]:
dataset = spec_0 + spec_1 + spec_2 + spec_3 + spec_4

## Validation Train Split

In [14]:
# Shuffle dataset
random.shuffle(dataset)

In [15]:
# Calculate a 20% test size
int(len(dataset) * 0.2)

353

In [16]:
# Split for training and testing (indicies are upto but not inclusive so we start at 331)
train = dataset[353:]
val = dataset[:353]

In [17]:
print('Training size:',len(train))
print('Validation size:',len(val))

Training size: 1414
Validation size: 353


## Pytorch Custom Dataset

In [18]:
# Build PyTorch dataset
class SpecDataset(data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        
    def transform(self,spec, label):
        spec = TF.to_tensor(spec)
        label = torch.tensor(label)
        return spec, label
        
    def __getitem__(self, index):
        spec = dataset[index][0]
        label = dataset[index][2]
        spec, label = self.transform(spec, label)
        return spec, label
    
    def __len__(self):
        return len(self.dataset)

# Test Dataset

In [38]:
# Create list of test measurements
test_measurements = []
for f in glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\subchallenge 1 test specs\*.png'):
    test_measurements.append(np.asarray(Image.open(f)))

In [39]:
# Prep the spectrograms
for i, spec in enumerate(test_measurements):
    test_measurements[i] = SpecPrep(spec)

In [40]:
# Create a list of measurement_ids
test_ids = list(map(os.path.basename,glob.iglob(r'C:\Users\Jonathan\Documents\AI Projects\BEAT PD\Data\subchallenge 1 test specs\*.png')))
for i, ID in enumerate(test_ids):
    test_ids[i] = os.path.splitext(ID)[0]

In [50]:
# Combine spectrograms and IDs as tuples
testdataset = []
for measurement, id_ in zip(test_measurements,test_ids):
    testdataset.append((measurement, id_))

# VGG16

In [22]:
# Load instance of VGG16
torch.manual_seed(314)
VGG16 = models.vgg16(pretrained=True)

In [23]:
# View final classifer section
VGG16.classifier[6]

Linear(in_features=4096, out_features=1000, bias=True)

In [24]:
# Turn of gradients for VGG16 parameters
for param in VGG16.parameters():
    param.requires_grad = False

In [25]:
# Modify last layer to have an output size of 5 (for our 5 classes)
torch.manual_seed(314)
VGG16.classifier[6] = nn.Sequential(nn.Linear(4096,512),nn.ReLU(),
                                    nn.Linear(512,5))

In [26]:
# View classifer section of VGG16
VGG16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Sequential(
    (0): Linear(in_features=4096, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=5, bias=True)
  )
)

# Lightning

In [34]:
class LightningVGG16(pl.LightningModule):

    def __init__(self, hparams, train_set, val_set):
        super(LightningVGG16, self).__init__()
        self.vgg16 = VGG16
        self.learning_rate = hparams.learning_rate
        self.train_batch_size = hparams.train_batch_size
        self.val_batch_size = hparams.val_batch_size
        self.train_set = train_set
        self.val_set = val_set
        
    def forward(self, x):
        x = self.vgg16(x)
        return x
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.vgg16.classifier[6].parameters(), lr=self.learning_rate)
    
    def training_step(self, train_batch, batch_idx):
        batch_corr = 0
        x, y = train_batch
        logits = self.forward(x)
        loss =  F.cross_entropy(logits, y)
        pred = logits.argmax(dim=1, keepdim=True)
        batch_corr += pred.eq(y.view_as(pred)).sum().item()
        acc = torch.tensor((batch_corr/self.train_batch_size) * 100)
        train_logs = {'training Loss': loss, 'Training Accuracy': acc, 'Number Correct in Training Batch': batch_corr}
        return {'loss': loss, 'Correct': batch_corr, 'acc': acc, 'log': train_logs}
    
    def validation_step(self, val_batch, batch_idx):
        batch_corr = 0
        x, y = val_batch
        logits = self.forward(x)
        loss = F.cross_entropy(logits, y)
        pred = logits.argmax(dim=1, keepdim=True)
        batch_corr += pred.eq(y.view_as(pred)).sum().item()
        acc = torch.tensor((batch_corr/self.val_batch_size) * 100)
        val_logs = {'Validation Loss': loss, 'Validation Accuracy': acc, 'Number Correct in Validation Batch': batch_corr}
        return {'val_loss': loss, 'val_acc': acc, 'log': val_logs}
        
    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        avg_acc = torch.stack([x['val_acc'] for x in outputs]).mean()
        tensorboard_logs = {'Average Validation Loss': avg_loss, 'Average Validation Accuracy': avg_acc}
        return {'val_loss': avg_loss, 'val_acc': avg_acc, 'log': tensorboard_logs}
        
    def prepare_data(self):
        self.prepped_trainset = SpecDataset(self.train_set)
        self.prepped_valset = SpecDataset(self.val_set)
        
    def train_dataloader(self):
        return DataLoader(self.prepped_trainset,batch_size=self.train_batch_size, shuffle=True)
    
    def val_dataloader(self):
        return DataLoader(self.prepped_valset,batch_size=self.val_batch_size, shuffle=True)

In [35]:
# Hyperparameters
hparams = Namespace(**{'learning_rate': 0.001, 'train_batch_size': 10,
                       'val_batch_size': 10})

In [36]:
# Run training/validation of model
model = LightningVGG16(hparams,train_set=train, val_set=val)
logger = TensorBoardLogger("lightning_logs", name="VGG16", version='Subchallenge 1')
trainer = pl.Trainer(early_stop_callback=True, max_epochs=10, gpus=1, logger=logger)
trainer.fit(model)

1

In [27]:
# Start tensorboard
notebook.start('--logdir lightning_logs --host localhost --port 6266')

ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 13676.

In [30]:
# Display tensorboard
notebook.display(port=6266)

Selecting TensorBoard with logdir lightning_logs (started 0:01:48 ago; port 6266, pid 1400).


In [ ]:
# List tensorboard instances
#notebook.list()

In [ ]:
#tensorboard kill
#!taskkill /pid 11468 /f

In [58]:
#torch.cuda.empty_cache()

# Predictions

In [60]:
# Load submission template
submission = pd.read_csv('Data/BEAT-PD_SC1_OnOff_Submission_Template.csv')
submission.head()

,measurement_id,prediction
0,0e80673b-8c90-474f-8388-00fdf25795e4,NaN
1,a7b61d40-9f26-4878-a2d7-bd766b8ae6f5,NaN
2,fc851afd-133c-4e8f-8deb-0079cc442358,NaN
3,90c2c320-1007-47b4-9f8e-cdd37067dfb2,NaN
4,379fd23b-e38f-4214-8a70-7a829004359f,NaN


In [71]:
# Predict and fill in submission template
for measurement, id_ in testdataset:
    output = model(torch.unsqueeze(TF.to_tensor(measurement).cuda(), dim=0))
    prediction = output.argmax(dim=1, keepdim=True).item()
    df_index = submission.index[submission['measurement_id'] == id_]
    submission.prediction[df_index] = prediction

In [84]:
# View predictions
pd.options.display.max_rows = 800
submission.head(n=800)

,measurement_id,prediction
0,0e80673b-8c90-474f-8388-00fdf25795e4,0
1,a7b61d40-9f26-4878-a2d7-bd766b8ae6f5,0
2,fc851afd-133c-4e8f-8deb-0079cc442358,0
3,90c2c320-1007-47b4-9f8e-cdd37067dfb2,0
4,379fd23b-e38f-4214-8a70-7a829004359f,0
5,6877b35f-474a-41b6-ab7b-d9fdcd3155ca,1
6,9e37d451-afba-494e-a531-e1c459c65a55,0
7,e94eee19-58a8-485e-b7e6-627ef92f5d32,0
8,22f6b2b7-d4e6-4066-a2de-0e2f3a62ee65,0
9,9afe4f12-18cf-4457-9f15-d90528b7d0e2,0


In [104]:
#Save submission df to CSV
submission.to_csv('Data/sc1_cis_submissions.csv', index=False)

In [14]:
class Foobar():
    def __init__(self, string):
        self.string = string
        
    def func1(self):
        self.string = 'ahhhh'
        
    def func2(stuff):
        stuff = 'ahhhh'
        return string + stuff

In [15]:
foo = Foobar('sup')

In [9]:
foo.func1()

In [10]:
foo.string

'ahhhh'

In [16]:
foo.func2()

NameError: name 'string' is not defined